# **Step 5: Utility-Based Promotional Targeting**

## **Objective:**
Identify customers for retention campaigns based on **expected incremental retention utility**. Prioritize interventions strategically, allocating resources effectively to maximize retention profit and minimize costs.

---

## ✅ **Theoretical Background:**

The core idea is to calculate the **Incremental Retention Utility** for each customer using:

$$
\text{Incremental Retention Utility} = (P_{\text{churn}} \times CLV) - \text{Intervention Cost}
$$

- **$P_{churn}$**: Predicted probability of churn (from the ML model)
- **$CLV$**: Customer Lifetime Value
- **Intervention Cost**: Cost of targeted promotions/interventions

---

### **1. Obtain churn probabilities**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from churn_model import train_cost_sensitive_churn_model

data = pd.read_csv('online_retail_customer_churn.csv')

# Encode categorical features (match training)
data_encoded = pd.get_dummies(data, columns=['Gender', 'Promotion_Response', 'Email_Opt_In'], drop_first=True)

# Feature Engineering (match training)
data_encoded['Return_Ratio'] = data_encoded['Num_of_Returns'] / data_encoded['Num_of_Purchases'].replace(0, 1)
data_encoded['Purchase_Frequency'] = data_encoded['Num_of_Purchases'] / data_encoded['Last_Purchase_Days_Ago'].replace(0, 1)
data_encoded['CLV'] = (data_encoded['Average_Transaction_Amount'] *
                       data_encoded['Purchase_Frequency'] *
                       data_encoded['Years_as_Customer']).fillna(0)

# Define intervention cost and utility explicitly (consistent with training)
intervention_cost = 10
data_encoded['Utility_True_Positive'] = data_encoded['CLV'] - intervention_cost
data_encoded['Utility_False_Positive'] = - intervention_cost
data_encoded['Utility_False_Negative'] = - data_encoded['CLV']
data_encoded['Utility_True_Negative'] = 0

# Extract features used during training
encoded_columns = [col for col in data_encoded.columns if 
                   col.startswith('Gender_') or
                   col.startswith('Promotion_Response_') or
                   col.startswith('Email_Opt_In_')]

features = ['Return_Ratio', 'Purchase_Frequency', 'CLV'] + encoded_columns

# Create X, y datasets
X = data_encoded[features]
y = data_encoded['Target_Churn'].astype(int)

# Split the data (consistent with your trained model)
X_train, X_test, y_train, y_test, util_train, util_test = train_test_split(
    X, y,
    data_encoded[['Utility_True_Positive', 'Utility_False_Positive', 
                  'Utility_False_Negative', 'Utility_True_Negative']],
    test_size=0.3, random_state=42
)

# Load the trained model explicitly
clf, _ = train_cost_sensitive_churn_model('online_retail_customer_churn.csv')

In [18]:
# Predict churn probabilities using the trained classifier from your function
churn_probabilities = clf.predict_proba(X_test)[:, 1]  # Probability of churn (class=1)

### **2. Calculate Incremental Retention Utility**

In [20]:
# Calculate Incremental Retention Utility for each customer in the test set
test_results = X_test.copy()
test_results['Churn_Probability'] = churn_probabilities
test_results['Actual_Churn'] = y_test.values
test_results['CLV'] = util_test['Utility_True_Positive'] + intervention_cost
test_results['Intervention_Cost'] = intervention_cost

test_results['Incremental_Utility'] = (test_results['Churn_Probability'] * test_results['CLV']) - test_results['Intervention_Cost']

### **3. Prioritize Customers for Promotional Targeting**

In [21]:
# Define thresholds for practical targeting (example)
high_value_threshold = test_results['CLV'].quantile(0.75)
moderate_risk_threshold = 0.4  # churn probability threshold
high_response_customers = data_encoded.loc[X_test.index, 'Promotion_Response_Responded'] == 1

# Practical Recommendations
def promotional_strategy(row):
    if (row['CLV'] >= high_value_threshold) and (row['Churn_Probability'] >= moderate_risk_threshold):
        return 'Target Aggressively'
    elif (row['CLV'] < high_value_threshold) and (row['Churn_Probability'] >= moderate_risk_threshold) and high_response_customers.loc[row.name]:
        return 'Moderate Targeting'
    else:
        return 'Minimal/No Targeting'

# Apply targeting strategy
test_results['Promotion_Strategy'] = test_results.apply(promotional_strategy, axis=1)

# Display the prioritized customers
print("Promotional Targeting Recommendations (top 10 examples):")
display(test_results[['Churn_Probability', 'CLV', 'Incremental_Utility', 'Promotion_Strategy']].head(10))

Promotional Targeting Recommendations (top 10 examples):


,Churn_Probability,CLV,Incremental_Utility,Promotion_Strategy
521,0.0,485.220950,-10.000000,Minimal/No Targeting
737,0.0,48.888348,-10.000000,Minimal/No Targeting
740,0.0,761.231244,-10.000000,Minimal/No Targeting
660,1.0,386.284472,376.284472,Minimal/No Targeting
411,1.0,2669.412857,2659.412857,Target Aggressively
678,1.0,67.322727,57.322727,Minimal/No Targeting
626,0.0,5065.028571,-10.000000,Minimal/No Targeting
513,1.0,244.069091,234.069091,Moderate Targeting
859,0.0,139.702909,-10.000000,Minimal/No Targeting
136,1.0,13.900073,3.900073,Moderate Targeting


## **Recommendations (Practical Examples Explained)**:

| Scenario                               | Recommended Action         | Explanation |
|----------------------------------------|----------------------------|-------------|
| **High CLV**, Moderate-high churn risk, Good engagement history | **Target aggressively**  | High financial value, responsive to promotions, cost-effective. |
| **Moderate CLV**, High churn risk, Previous positive promotion response | **Moderate targeting**   | Worth targeting due to reasonable likelihood of response, manageable risk/cost. |
| **Low CLV**, High churn risk, Low response likelihood | **Minimal/no targeting** | Intervention costs outweigh benefits, low expected return on investment. |

---

## **Expected Outcomes:**
- Improved customer retention rate.
- Efficient use of marketing resources (reducing wasted spend).
- Increased overall profitability by prioritizing high-value, responsive customers.

---

## **Final Recommendations for Implementation:**
- **Regularly update churn predictions** (monthly/quarterly) based on latest data.
- Continuously measure and adapt **intervention costs** based on marketing effectiveness.
- Adjust thresholds (CLV, churn probability, engagement history) as real-world data informs effectiveness.